# 0. Initialize

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


## 0.1. Import Libraries

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

## 0.2. DEFINE VARIABLES 

In [88]:
DATA_PATH = '/gdrive/MyDrive/412term/raw_data/' # '<insert-your-training-data-path-here>'
batupath='/gdrive/MyDrive/412term/'

ROUND = 3 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '26851'#'<insert-your-id-here>'
PROJECT_CODE = 'CS412d9691e60bd94'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [4]:
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})

trainingUserDfb = pd.read_csv('{}batuhan_tweets.csv'.format(batupath), dtype={'tweet_id': str, 'isPolitical': str})
#trainingUserDfm = pd.read_csv('{}emre_tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
#trainingUserDfy = pd.read_csv('{}yagiz_tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
#trainingUserDfe = pd.read_csv('{}mete_tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
#trainingUserDfb = pd.concat([trainingUserDfb, trainingUserDfy])
#trainingUserDfb= pd.concat([trainingUserDfb, trainingUserDfm])
#trainingUserDfb= pd.concat([trainingUserDfb, trainingUserDfe])

columns = trainingUserDfb.columns.tolist()
columns[0] = "tweet_id"
trainingUserDfb.columns = columns

last=trainingUserDfb[['tweet_id','isPolitical']]


last['tweet_id']= last['tweet_id'].astype(str)

trainingTweetDf = pd.concat([trainingTweetDf,last])

trainingTweetDf = trainingTweetDf.drop_duplicates(subset=['tweet_id'])

trainingTweetDf

<ipython-input-4-c6d448905144>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last['tweet_id']= last['tweet_id'].astype(str)


,tweet_id,isPolitical
0,1597170281545551872,Yes
1,1431700027471192069,No
2,1566035577090281472,Yes
3,1591538690869940225,Yes
4,1583898169238167554,Yes
...,...,...
195,1588176213951946754,Yes
196,1393473091309817858,No
197,1588915997859545089,Yes
198,1588948740651749376,Yes


In [5]:
trainingTweetDf.isPolitical.value_counts()

Yes    2134
No     1037
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [6]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
#trainingUserDf = pd.read_csv('training-user.csv')

UserDfb = pd.read_csv('{}batuhan_users.csv'.format(batupath), dtype={'tweet_id': str, 'isPolitical': str})
UserDfm = pd.read_csv('{}emre_user.csv'.format(batupath), dtype={'tweet_id': str, 'isPolitical': str})
UserDfy = pd.read_csv('{}yagiz_user.csv'.format(batupath), dtype={'tweet_id': str, 'isPolitical': str})
UserDfe = pd.read_csv('{}mete_user.csv'.format(batupath), dtype={'tweet_id': str, 'isPolitical': str})
UserDfb = pd.concat([UserDfb, UserDfy])
UserDfb= pd.concat([UserDfb, UserDfm])
UserDfb= pd.concat([UserDfb, UserDfe])

UserDfb.rename(columns = {'Unnamed: 0':'screen_name'}, inplace = True)
last=UserDfb[['screen_name','isBot']]

trainingUserDf = pd.concat([trainingUserDf,last])

trainingUserDf = trainingUserDf.drop_duplicates(subset=['screen_name'])

trainingUserDf

,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
193,metyus34,No
194,mehmettemiz38,Yes
195,agreedont,Not sure
196,huseyindemir61,No


In [7]:
trainingUserDf.isBot.value_counts()

No          2784
Yes          730
Not sure      53
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [8]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS412d9691e60bd94.html


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [9]:
PATH_TO_DOWNLOADED = DATA_PATH # 'https://drive.google.com/drive/folders/1hu5E-6MFHsBOnpI1G6Ble2njPU__ZwCJ?usp=share_link'

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data

#### 1.1.1.1. Check if Retweet

In [10]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.2. Get Tweet Text

In [11]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text

#### 1.1.1.3. Get Tweet ID

In [12]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

In [13]:
def get_listed_count(tweet_metadata_line):
    listed_count = tweet_metadata_line['user']['listed_count']
    
    return listed_count

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [14]:
def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])

    return num_mentions, num_hashtags

#### 1.1.1.5. Get Number of Retweets and Favorites

In [15]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.6. Get User Info

In [16]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

In [17]:
def check_if_user_bot(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [18]:
def check_political_ent(text):
    
    # the list below can be modified and some new names may be added (or removed)
    list_of_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag','suleymansoylu','Ahmet_Davutoglu','alibabacan','ekrem_imamoglu','ansuryavas06','BY','herkesicinCHP','Akparti','dbdevletbahceli','iyiparti','MHP_Bilgi','zaferpartisi','tcbestepe','GelecekPartiTR','SaadetPartisi','drfahrettinkoca','saglikbakanligi','tcmeb','TC_Disisleri','tcsavunma','ziyaselcuk','TC_icisleri','TCSanayi','TCTarim','ticaret','UABakanligi','TBMMresmi','atama','milli','bekliyoruz','cumhuriyet','millet','parti','recep','mansur','ekrem','tayyip','fahrettin','adalet','eyt']
    
    entities_in_text = [ent for ent in list_of_entities if ent.lower() in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

#### 1.1.2.2. Number of total interactions

In [19]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

### 1.1.2. Collect data using the functions above and transform into a Pandas DataFrame

In [20]:
dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'num_political_entities':[],
              'total_interactions':[],
              'listed_count':[],
               
               }


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text = get_tweet_text(line)
        num_mentions, num_hashtags = get_number_mentions_hashtags(line)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)
        
        
       

        # manually crafted data:
        num_political_entities = check_political_ent(text)
        total_num_interactions = total_interactions(retweet_count, favorite_count)
        listed_count = get_listed_count(line)

        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['num_political_entities'].append(num_political_entities)
        dfPolitical['total_interactions'].append(total_num_interactions)
        dfPolitical['listed_count'].append(listed_count)

In [21]:
dfPolitical = pd.DataFrame(dfPolitical)

dfPolitical

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,listed_count
0,1588568792984346624,0,None,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,0
1,1588452263047069697,0,None,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,,1,0,3
2,1569589330544398336,0,None,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,0
3,1570428119609139201,0,None,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,0
4,1551163840368414722,0,None,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,0,None,Gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,9,81,576247173,ardanzenturk,RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...,0,90,379
33529,1584027427696959488,0,None,@umitozdag Neden Suriyelilerle ilgili bu kadar...,1,0,1,8,162308585,ozgul_61,Bridge design engineer Yaay hesabı : dilfiruz,1,9,4
33530,1585945783307730945,0,None,@celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...,1,0,0,1,415025519,ladrekova,,1,1,0
33531,1569748909521801221,1,muazzezeralp,RT @muazzezeralp: @Doan58213655 @denizkonur @N...,7,1,6,0,1442125177727307781,yapikytgrivrlsn,,1,6,0


## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

#### 1.2.1.1. Get user info metadata

In [22]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    is_verified = user_metadata_line['verified']
    default_profile_image = user_metadata_line['default_profile_image']
    followers_count = user_metadata_line['followers_count']
    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count,'is_verified':is_verified,'default_profile_image':default_profile_image,'followers_count':followers_count}

    return dictionary

In [23]:
def check_if_verified(tweet_metadata_line):
    is_verified = 0

  
    try:
        tweet_metadata_line['user']
        is_verified = tweet_metadata_line['user']['verified'].lower()
        
        is_verified = 1

    except KeyError:
        pass

    return is_verified

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [24]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

#### 1.2.1.3. Get description length

In [25]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [26]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'description_len':[],
         'followers_to_all_ratio':[],
         'is_verified':[],
         'default_profile_image':[],
         'followers_count':[]
         }

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

In [27]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,is_verified,default_profile_image,followers_count
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,False,False,65
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,False,False,8235
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,False,False,95
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,False,False,40
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,False,False,100
...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,False,False,229
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,False,False,212457
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,False,False,116
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,False,False,1669


### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [28]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

#### 1.2.2.2. Check median number of favorites

In [29]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [30]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[]
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))

        i += 1
        if i % 1000 == 0:
            print(i)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [31]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets
0,594642154,0.115000,2.0,200
1,525600289,0.005025,1.0,199
2,931895965501534209,0.900000,0.0,200
3,1591543462746329088,0.185000,0.0,200
4,734801354749796352,1.000000,0.0,200
...,...,...,...,...
28310,1591370361488252928,0.800000,0.0,200
28311,1475272459616235525,0.825000,0.0,200
28312,1096753792731750401,0.051020,1.0,196
28313,1269527617687953409,0.095000,2.0,200


### 1.2.3. Merge dfBot and dfBotTweets

In [32]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)

dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,is_verified,default_profile_image,followers_count,retweet_total_ratio,num_median_favorites,num_of_tweets
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,False,False,65,0.395939,0.0,197.0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,False,False,8235,0.125000,0.0,200.0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,False,False,95,0.910000,0.0,200.0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,False,False,40,0.015306,1.0,196.0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,False,False,100,0.659898,0.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,False,False,229,0.015000,1.0,200.0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,False,False,212457,0.291457,2.0,199.0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,False,False,116,0.061538,0.0,195.0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,False,False,1669,0.995000,0.0,200.0


# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [33]:
dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf,
                                         on='tweet_id')

dfPoliticalAll_train

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,listed_count,isPolitical
0,1588568792984346624,0,None,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,0,Yes
1,1588452263047069697,0,None,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,,1,0,3,Yes
2,1569589330544398336,0,None,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,0,Yes
3,1570428119609139201,0,None,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,0,Yes
4,1551163840368414722,0,None,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,1565798698029924353,0,None,@dbdevletbahceli Şoför,1,0,0,0,1373715344301391882,ahmetazrak8,,1,0,0,No
3170,1586019337248309248,0,None,Türkiyenin geleceği ücretli öğretmenliğe emane...,4,0,0,0,1583586562817785860,taskinali003,,1,0,0,Yes
3171,1589888339108585474,0,None,@enginozkoc ARTIK YETER !\nSUÇ DUYURUSUDUR\n@R...,3,0,0,0,1589554549790052353,00can02,,2,0,0,Yes
3172,1587113229041508352,0,None,@mahirunal Parti adına doğrusunu yaptın.,1,0,0,0,1491090669263278081,teomantiryakio2,,1,0,0,Yes


### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [34]:
print(len(dfPoliticalAll_train))
X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags','num_mentions','num_retweets','num_favorites','listed_count']]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

3174


### 2.1.3. Train - validation split

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import GridSearchCV

dtc_political = DecisionTreeClassifier()

param_grid = {'max_depth':[3,5,7,9,11]} 
grid = GridSearchCV(dtc_political,
                    param_grid, 
                    cv=5,
                    scoring='f1',
                    return_train_score=False, 
                    verbose=1, 
                    refit=True)

dct_grid_search = grid.fit(X_train, y_train)

# fit your model
dtc_political.fit(X_train, y_train)
# make predictions
y_pred = dct_grid_search.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, y_pred)
confusion = confusion_matrix(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)


print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
MSE: 0.22992125984251968 
 Accuracy Score: 0.7700787401574803 
 Confusion Matrix: 
 [[ 81 122]
 [ 24 408]]


In [37]:
from xgboost import XGBClassifier

# read in data
classifierxgb = XGBClassifier()

param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [ 0.8, 1.0],
        'max_depth': [3]
        }



grid = GridSearchCV(classifierxgb,
                    param_grid, 
                    cv=5,
                    return_train_score=False, 
                    verbose=1, 
                    refit=True,
                    n_jobs = -1)

xgb_grid_search = grid.fit(X_train, y_train)

y_pred = xgb_grid_search.predict(X_valid)

acc_score = accuracy_score(y_valid, y_pred)
confusion = confusion_matrix(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)



print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


KeyboardInterrupt: ignored

In [38]:
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras_preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer 
from keras.layers import Bidirectional
from keras import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras.layers import concatenate

In [39]:
seq_length = 128
vocab_size = 250
embedding_size = 24
classifier_neurons = embedding_size + len(X_train.columns)
classifier_neurons

31

In [40]:
max_len = max(dfPoliticalAll_train['text'], key=len)
print(len(max_len))

152


In [47]:
X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags','num_mentions','num_retweets','num_favorites','listed_count','text']]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1. if x=='Yes' else 0.)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

In [48]:
X_meta_train = X_train[['num_political_entities','total_interactions','num_hashtags','num_retweets','num_mentions','num_favorites']]
X_text_train = [one_hot(d, vocab_size) for d in X_train['text']]

X_meta_valid = X_valid[['num_political_entities','total_interactions','num_hashtags','num_retweets','num_mentions','num_favorites']]
X_text_valid = [one_hot(d, vocab_size) for d in X_valid['text']]

In [49]:
padded_text_train = pad_sequences(X_text_train, maxlen=seq_length, padding='post')
padded_text_valid = pad_sequences(X_text_valid, maxlen=seq_length, padding='post')

In [50]:
nlp_input = Input(shape=(seq_length,), name='nlp_input')
meta_input = Input(shape=(6,), name='meta_input')
emb = Embedding(output_dim=embedding_size, input_dim=vocab_size, input_length=seq_length)(nlp_input)
nlp_out = Bidirectional(LSTM(32, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=regularizers.l2(0.01)))(emb)
x = concatenate([nlp_out, meta_input])
x = Dense(classifier_neurons, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[nlp_input , meta_input], outputs=[x])

In [51]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 nlp_input (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 128, 24)      6000        ['nlp_input[0][0]']              
                                                                                                  
 bidirectional (Bidirectional)  (None, 64)           14592       ['embedding[0][0]']              
                                                                                                  
 meta_input (InputLayer)        [(None, 6)]          0           []                               
                                                                                              

In [52]:
model.fit({'nlp_input': padded_text_train, 'meta_input': X_meta_train}, y_train, 
          validation_data=({'nlp_input': padded_text_valid, 'meta_input': X_meta_valid}, y_valid),
          epochs=15, batch_size=128)

Epoch 1/15
20/20 [==============================] - 19s 582ms/step - loss: 0.9708 - val_loss: 0.8266
Epoch 2/15
20/20 [==============================] - 8s 380ms/step - loss: 0.7327 - val_loss: 0.6383
Epoch 3/15
20/20 [==============================] - 7s 361ms/step - loss: 0.5685 - val_loss: 0.4974
Epoch 4/15
20/20 [==============================] - 8s 384ms/step - loss: 0.4510 - val_loss: 0.4030
Epoch 5/15
20/20 [==============================] - 7s 353ms/step - loss: 0.3699 - val_loss: 0.3331
Epoch 6/15
20/20 [==============================] - 7s 343ms/step - loss: 0.3119 - val_loss: 0.2874
Epoch 7/15
20/20 [==============================] - 7s 326ms/step - loss: 0.2707 - val_loss: 0.2506
Epoch 8/15
20/20 [==============================] - 7s 355ms/step - loss: 0.2401 - val_loss: 0.2222
Epoch 9/15
20/20 [==============================] - 7s 334ms/step - loss: 0.2155 - val_loss: 0.2380
Epoch 10/15
20/20 [==============================] - 7s 333ms/step - loss: 0.1969 - val_loss: 0.186

In [53]:
score = model.evaluate({'nlp_input': padded_text_valid, 'meta_input': X_meta_valid}, y_valid, verbose=0)
print('Eval loss:', score)

Eval loss: 0.16637863218784332


In [54]:
preds = model.predict({'nlp_input': padded_text_valid, 'meta_input': X_meta_valid})
preds = [x[0] for x in preds ]

20/20 [==============================] - 1s 29ms/step


In [55]:
max(preds), min(preds)

(0.99993676, 5.780941e-22)

In [56]:
y_pred = [0 if x < 0.5 else 1 for x in preds]
mse = mean_squared_error(y_valid, y_pred)
print("MSE:", mse, "\n")

y_pred = preds
mse = mean_squared_error(y_valid, y_pred)
print("MSE:", mse, "\n")

MSE: 0.2251968503937008 

MSE: 0.16096004501450184 



In [57]:
dfPolitical.head

<bound method NDFrame.head of                   tweet_id  is_retweet retweeted_username  \
0      1588568792984346624           0               None   
1      1588452263047069697           0               None   
2      1569589330544398336           0               None   
3      1570428119609139201           0               None   
4      1551163840368414722           0               None   
...                    ...         ...                ...   
33528  1568595408233832448           0               None   
33529  1584027427696959488           0               None   
33530  1585945783307730945           0               None   
33531  1569748909521801221           1       muazzezeralp   
33532  1585926877675282432           0               None   

                                                    text  num_mentions  \
0      Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...             0   
1      @mahirunal Gavur İzmir ya onlar, hani Cumhuriy...             1   
2      #ŞehitAd

In [62]:
testTweetDf = trainingTweetDf
testTweetDf.dropna(inplace=True)
columns = testTweetDf.columns.tolist()
columns[0] = "tweet_id"
testTweetDf.columns = columns
testTweetDf["tweet_id"]= testTweetDf["tweet_id"].astype(str)
dfPoliticalAll_test= dfPolitical.merge(testTweetDf,
                                         on='tweet_id')

X_meta_test = dfPoliticalAll_test[['num_political_entities','total_interactions','num_hashtags','num_retweets','num_mentions','num_favorites']]
X_text_test = [one_hot(d, vocab_size) for d in dfPoliticalAll_test['text']]
dfPoliticalAll_test.head
y_test = dfPoliticalAll_test['isPolitical'].apply(lambda x: 1. if x=='Yes' else 0.)
padded_text_test = pad_sequences(X_text_test, maxlen=seq_length, padding='post')

In [63]:
dfPoliticalAll_test.head

<bound method NDFrame.head of                  tweet_id  is_retweet retweeted_username  \
0     1588568792984346624           0               None   
1     1588452263047069697           0               None   
2     1569589330544398336           0               None   
3     1570428119609139201           0               None   
4     1551163840368414722           0               None   
...                   ...         ...                ...   
3166  1565798698029924353           0               None   
3167  1586019337248309248           0               None   
3168  1589888339108585474           0               None   
3169  1587113229041508352           0               None   
3170  1596906607971307522           0               None   

                                                   text  num_mentions  \
0     Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...             0   
1     @mahirunal Gavur İzmir ya onlar, hani Cumhuriy...             1   
2     #ŞehitAdayıUzmÇvşaKadro\

In [64]:
score = model.evaluate({'nlp_input': padded_text_test, 'meta_input': X_meta_test}, y_test, verbose=0)
print('Eval loss:', score)

Eval loss: 0.1581423431634903


In [65]:
mse = mean_squared_error(y_valid, y_pred)
print(mse)

0.16096004501450184


## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [66]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [67]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,is_verified,default_profile_image,followers_count,retweet_total_ratio,num_median_favorites,num_of_tweets,screen_name,isBot
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,False,False,65,0.395939,0.0,197.0,nasreenakhan006,No
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,False,False,8235,0.125000,0.0,200.0,scorpiehoez,Yes
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,False,False,95,0.910000,0.0,200.0,yusufak63712920,Not sure
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,False,False,40,0.015306,1.0,196.0,nedenburdaysam,No
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,False,False,100,0.659898,0.0,197.0,biologselim,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565,1286770207134973954,Hamide Arabacı,anka6054,,,151,61,0,0.712264,False,False,151,0.000000,1.0,200.0,anka6054,No
3566,1598032338323214338,atamabekleyenbahceci,atamabekleyenzz,,,173,367,0,0.320370,False,False,173,0.580000,0.0,200.0,atamabekleyenzz,No
3567,760235343966863360,Emrah İNCİ,memrahinci,Istanbul - Bayburt,Researcher | Middle East | Political Science |...,5863,5905,71,0.498215,False,False,5863,0.040000,36.0,200.0,memrahinci,No
3568,1553973684100124672,Murat Kkk,muratkkk18,,Normal sıradan bir insanım,1,10,26,0.090909,False,False,1,0.769231,0.0,13.0,muratkkk18,No


In [68]:
trainingUserDf.isBot.value_counts()

No          2784
Yes          730
Not sure      53
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [69]:
X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio','is_verified']]
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

### 2.2.3. Train-test split

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.2.4. Train the model

In [71]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import GridSearchCV

dtc_bot = DecisionTreeClassifier()

param_grid = {'max_depth':[3,5,7,9,11]} 
grid = GridSearchCV(dtc_bot,
                    param_grid, 
                    cv=5,
                    scoring='f1',
                    return_train_score=False, 
                    verbose=1, 
                    refit=True)

dct_grid_search = grid.fit(X_train, y_train)

# fit your model
dtc_bot.fit(X_train, y_train)
# make predictions
y_pred = dct_grid_search.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, y_pred)
confusion = confusion_matrix(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
MSE: 0.19187675070028012 
 Accuracy Score: 0.8081232492997199 
 Confusion Matrix: 
 [[548  23]
 [114  29]]


# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [74]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('{}evaluation-round{}-tweet.csv'.format(DATA_PATH,ROUND), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags','num_mentions','num_retweets','num_favorites','listed_count','text']]


# make predictions based on these variables

predictions_political = model.predict({'nlp_input': padded_text_test, 'meta_input': X_meta_test})

100/100 [==============================] - 4s 44ms/step


### This part is important! We expect you to return your predictions in the following format:

In [75]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1434787703783051264': 0.9984856247901917,
 '1367571642604544000': 0.6369613409042358,
 '1589993032975544320': 0.5247973799705505,
 '1565312596135354373': 0.7945064306259155,
 '1579558096833511424': 0.632802426815033,
 '1439547067337256967': 0.5096233487129211,
 '1559963768372740098': 0.7929270267486572,
 '1562853131251118081': 0.5534937977790833,
 '1586021183958704128': 0.36734676361083984,
 '1585766233491886081': 0.513344407081604,
 '1427746815420604417': 0.3321428894996643,
 '1352635736537882629': 0.32381024956703186,
 '1415032260571680768': 0.9290417432785034,
 '1548636597628899328': 0.9517462253570557,
 '1564926450096013313': 0.8547157049179077,
 '1585634359612420101': 0.8815834522247314,
 '1597138789108895744': 0.5760055184364319,
 '1391681495622995971': 0.2654019892215729,
 '1389951943343316995': 0.9550942182540894,
 '1452348722810138646': 0.07753515243530273,
 '1595829502021623812': 0.5540311336517334,
 '1413108476348354562': 0.263629287481308,
 '1579408398894137344': 0.704763

## 3.2. Predictions for Users (Bot or Not)

In [76]:
evaluationUserDf = pd.read_csv('{}evaluation-round{}-user.csv'.format(DATA_PATH,ROUND), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio','is_verified']]

# make predictions based on these variables
predictions_bot = dtc_bot.predict(X)

In [77]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'biologselim': 0.0,
 'omerakdag34': 0.0,
 'bilgin21604923': 0.0,
 '_sydneycarton_': 0.0,
 'denizlihabercom': 0.0,
 'burakerbaychp': 0.0,
 'mvnez': 1.0,
 'qara118': 0.0,
 'nabiyonyevrum': 0.0,
 'farukhalit2': 0.0,
 'harlunoshi': 0.0,
 'heritagepaix': 0.0,
 'nuranwolf': 0.0,
 'politikgundem': 1.0,
 'isakethudax': 0.0,
 'enveraysevera': 0.0,
 'ilaydejaneiro': 0.0,
 '1905anason': 1.0,
 'eraydurgut03': 0.0,
 'dasiskein': 0.0,
 'ercan_bas29': 0.0,
 'mett_1907': 0.0,
 'ondemir066': 0.0,
 'semihyeteer': 0.0,
 'haberinyokcokk': 0.0,
 'meleky_ozaydin': 0.0,
 'mehmetaltay64': 0.0,
 'nurtencam2': 0.0,
 'bilobi4': 0.0,
 'yorumsuzadam87': 1.0,
 'twitsildiren': 0.0,
 '21gramlife1': 1.0,
 'cakan0_': 0.0,
 'oguzksalici': 0.0,
 'canpixlr': 0.0,
 'ozlemesor': 0.0,
 'emre_caliskann': 0.0,
 'oguzhanonel': 0.0,
 'mintchocolatemx': 0.0,
 'mehmet07454846': 0.0,
 'begumkarabeyx': 0.0,
 'imamgibiimam': 0.0,
 'durdane52': 0.0,
 'rk_ozanyali': 0.0,
 'suaterce': 0.0,
 'ibrahimcalsskan': 0.0,
 'goksubayy': 0.0,
 '

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [82]:
# Explain your approach

data_explanations = '''
Firstly, we get our individual data and combined all of them, then we select only 2 columns of the data.
Then we combined the new data with the given data. We did the same process for both tweets and users data.
Then we dropped the duplicate rows from the resulting data.
'''

feature_explanations = '''
we extracted the verified part from the user section (tweet_metadata.jsons.gz) and added to our model which resulted as a increased accuracy'
we extracted even more features for political tweet prediction since we were not getting the improvements we were hoping to get.
'''

model_explanations = '''
We used decision tree model for both tweet and user data and we used grid_search_cv() to optimize the hyperparameters of your model

'''

additional_explanations = '''
we have used nlp in political tweet prediction
'''


In [89]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('{}predictions-{}_round{}.json'.format(batupath,STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [90]:
# Test your submission file

submission = json.load(open('{}predictions-{}_round{}.json'.format(batupath,STUDENT_ID, ROUND), 'r'))
submission

{'round': 3,
 'student_id': '26851',
 'user_predictions': {'biologselim': 0.0,
  'omerakdag34': 0.0,
  'bilgin21604923': 0.0,
  '_sydneycarton_': 0.0,
  'denizlihabercom': 0.0,
  'burakerbaychp': 0.0,
  'mvnez': 1.0,
  'qara118': 0.0,
  'nabiyonyevrum': 0.0,
  'farukhalit2': 0.0,
  'harlunoshi': 0.0,
  'heritagepaix': 0.0,
  'nuranwolf': 0.0,
  'politikgundem': 1.0,
  'isakethudax': 0.0,
  'enveraysevera': 0.0,
  'ilaydejaneiro': 0.0,
  '1905anason': 1.0,
  'eraydurgut03': 0.0,
  'dasiskein': 0.0,
  'ercan_bas29': 0.0,
  'mett_1907': 0.0,
  'ondemir066': 0.0,
  'semihyeteer': 0.0,
  'haberinyokcokk': 0.0,
  'meleky_ozaydin': 0.0,
  'mehmetaltay64': 0.0,
  'nurtencam2': 0.0,
  'bilobi4': 0.0,
  'yorumsuzadam87': 1.0,
  'twitsildiren': 0.0,
  '21gramlife1': 1.0,
  'cakan0_': 0.0,
  'oguzksalici': 0.0,
  'canpixlr': 0.0,
  'ozlemesor': 0.0,
  'emre_caliskann': 0.0,
  'oguzhanonel': 0.0,
  'mintchocolatemx': 0.0,
  'mehmet07454846': 0.0,
  'begumkarabeyx': 0.0,
  'imamgibiimam': 0.0,
  'du